# microbit IoT-knoop

We beschrijven hier het gebruik van een microbit als IoT-knoop.
De communicatie van de sensor- en actuator-waarden is gebaseerd op het Cayenne LPP (Low Power Payload) formaat.
Voor het coderen en decoderen van dat formaat gebruiken we de module `ulpp`. (Zie elders in dit boek.)

Voor de communicatie gebruiken we het volgende formaat:

* protocol-tag (1 byte, =0x0A voor uplink, 0x0B voor downlink)
* node-ID (2 bytes)
* counter (2 bytes)
* sensor/actuator data (in LPP formaat)

Dit formaat wordt gebruikt voor de berichten van de IoT-knoop naar de gateway (uplink), als omgekeerd (downlink).

Het microbit radio-protocol gebruikt geen adressering: elk bericht is een broadcast in het netwerk.
We gebruiken de node-ID voor de adressering: een microbit IoT-knoop moet van elk binnenkomend bericht in software controleren of dat voor deze knoop bestemd is.

Met het protocol-tag onderscheiden we de IoT-berichten van de **microbit tekst-berichten**: deze beginnen met de header 0x01, 0x00, 0x01, gevolgd door een string-waarde.

In [1]:
from microbit import *
from ulpp import LppFrame, bytes_to_dict
import radio
import machine
from utime import *

def get_serial_number() -> int:
    NRF_FICR_BASE = 0x10000000
    DEVICEID_INDEX = 25 # deviceid[1]
    return machine.mem32[NRF_FICR_BASE + (DEVICEID_INDEX*4)] & 0xFFFFFFFF

nodeID = get_serial_number() & 0xFFFF
counter = 0

uplink_tag = 0x0A
downlink_tag = 0x0B

led0 = 0
led1 = 0 # not used

def handle_actuators(data):
    lpp = bytes_to_dict(data)
    if 0 in lpp and 'dOut' in lpp[0]:
        led0 = lpp[0]['dOut']
        pin0.write_digital(led0)
        if led0 == 1:
            display.show(Image.HAPPY)
        else:
            display.show(Image.SAD)
    if 1 in lpp and 'dOut' in lpp[1]:
        led1 = lpp[1]['dOut']
        pin1.write_digital(led0)

def send_sensors(buttonA, buttonB):
    global counter
    
    print("send")
    (chi, clo) = divmod(counter, 256)
    (nhi, nlo) = divmod(nodeID, 256)
    prefix = bytes([uplink_tag, nhi, nlo, chi, clo])
    counter += 1
  
    frame = LppFrame(data=prefix)
    frame.add_digital_output(0, led0)
    frame.add_digital_output(1, led1)
    frame.add_digital_input(2, buttonA)
    frame.add_digital_input(3, buttonB)
    frame.add_temperature(4, temperature()*10)
    frame.add_analog_input(8, display.read_light_level())
    buffer = frame.to_bytes()
    radio.send_bytes(buffer) 
    
timer_period = 60000
timer_deadline = ticks_add(ticks_ms(), timer_period)

display.scroll(hex(nodeID))
print(hex(nodeID))

radio.on()
send_sensors(button_a.is_pressed(), button_b.is_pressed())
while True:
    if button_a.was_pressed():
        send_sensors(1, button_b.is_pressed())
        
    if button_b.was_pressed():
        send_sensors(button_a.is_pressed(), 1)
        
    if ticks_diff(timer_deadline, ticks_ms()) <= 0:
        send_sensors(button_a.is_pressed(), button_b.is_pressed())
        timer_deadline = ticks_add(timer_deadline, timer_period)
        
    rec_bytes = radio.receive_bytes()
    if rec_bytes != None:
        if rec_bytes[0] == downlink_tag:
            rec_nodeID = rec_bytes[1] * 256 + rec_bytes[2]
            if nodeID == rec_nodeID:  # msg for this node
                display.show('R')
                rec_counter = rec_bytes[3] * 256 + rec_bytes[4]
                handle_actuators(rec_bytes[5:])
                send_sensors(button_a.is_pressed(), button_a.is_pressed())

    sleep_ms(10)

ModuleNotFoundError: No module named 'microbit'

## Identificatie van een microbit-knoop

Als identificatie voor een IoT-knoop gebruiken we een 16-bits getal, in JSON en in MQTT topics weergegeven als hexadecimale string.

> In het geval van WiFi-knopen gebruiken we daarvoor de laatste 16 bits van het WiFi MAC adres.

Een microbit heeft een unieke identificatie: we kunnen daarvan de laatste 16 bits gebruiken.

```
from microbit import *
import machine

def get_serial_number(type=hex):
    NRF_FICR_BASE = 0x10000000
    DEVICEID_INDEX = 25 # deviceid[1]
    return type(machine.mem32[NRF_FICR_BASE + (DEVICEID_INDEX*4)]& 0xFFFFFFFF)
```

In [7]:
hex(20000)[-4:]

'4e20'

NB: in de LPP-communicatie gebruiken we de identificatie als getal!

NB:

* er is een verschil tussen het decoderen van binaire LPP in een IoT-knoop, met als doel het aansturen van de actuatoren; en het decoderen van binaire LPP in de gateway, met als doel de omzetting in JSON. In dit laatste geval moet je alle elementen decoderen, in het eerste geval alleen maar de actuatoren.
* bij het decoderen in de IoT-knoop moet de inhoud overeenkomen met de actuele actuatoren van de IoT-knoop. Hoe controleer je dit?
    * het channel bepaalt al met welke actuator je te maken hebt. 
* worden de sensoren geskipt?
* in de IoT-knoop heb je (in principe) alleen te maken met gehele getallen als waarden voor de sensoren en actuatoren.


Decoderen van LPP in een IoT-knoop, omzetten in actuator-acties:

In [ ]:
in_buffer = bytearray([])
in_pos = 0

def get_byte() -> int:
    global in_pos
    byte = in_buffer[in_pos]
    in_pos += 1
    return byte

def get_int() -> int:
    hi = get_byte()
    lo = get_byte()
    return hi * 256 + lo

# NB: onderstaande functies zijn niet generiek,
# maar specifiek voor de IoT-knoop.

def handle_led(channel: int, value: int):
    if channel == 0:
        pin0.write_digital(value)
        if value == 0:
            display.show(off_image)
        else:
            display.show(on_image)
    elif channel == 1:
        pin1.write_digital(value)

def handle_lpp_actuators():  # naam?
    while in_pos < len(in_buffer):
        channel = get_byte()
        tag = get_byte()
        if (channel == 0 or channel == 1) and tag == LPP_dOut:
            handle_led(channel, get_byte())
        elif tag == LPP_dIn or tag == LPP_presence or tag == LPP_humidity:
            skip = get_byte()
        else:
            skip = get_int()

Coderen van LPP in IoT-knoop.

Ook dit hoeft niet helemaal generiek te zijn: je hoeft alleen voor de aanwezige sensoren (en actuatoren?) te coderen.

In [ ]:
out_buffer = bytearray([])
out_pos = 0

def put_byte(value: int):
    out_buffer[out_pos] = value
    out_pos += 1

def put_int(value: int):
    while value < 0:
        value = value + 65536
    put_byte(value // 256)
    put_byte(value % 256)

def put_sensor_byte(channel: int, tag: int, value: int):
    put_byte(channel)
    put_byte(tag)
    put_byte(value)
    
def put_sensor_int(channel: int, tag: int, value: int)
    put_byte(channel)
    put_byte(tag)
    put_int(value)

def send_sensors(button0: int, button1:int, presence: int):
    put_sensor_byte(0, LPP_dOut, pin0.read_digital()) # or: led0
    put_sensor_byte(1, LPP_dOut, pin1.read_digital()) # or: led1
    put_sensor_byte(2, LPP_dIn, button0)
    put_sensor_byte(3, LPP_dIn, button1)
    put_sensor_int(4, LPP_temperature, machine.temperature() * 10)
    put_sensor_byte(7, LPP_presence, presence)
    put_sensor_int(8, LPP_aOut, display.read_light_level())

In [ ]:
from microbit import *
import radio
import machine
from utime import *

# version without LPP library
# LPP tags
LPP_dIn = 0
LPP_dOut = 1
LPP_aIn = 2
LPP_aOut = 3
LPP_luminosity = 101
LPP_presence = 102
LPP_temperature = 103
LPP_humidity = 104
LPP_barometer = 115

def get_serial_number() -> int:
    NRF_FICR_BASE = 0x10000000
    DEVICEID_INDEX = 25 # deviceid[1]
    return machine.mem32[NRF_FICR_BASE + (DEVICEID_INDEX*4)] & 0xFFFFFFFF

nodeID = get_serial_number() & 0xFFFF
counter = 0

uplink_tag = 0x0A
downlink_tag = 0x0B

led0 = 0
led1 = 0

def handle_led(channel: int, value: int):
    global led0, led1
    if channel == 0:
        led2 = value
        pin0.write_digital(value)
        if value == 0:
            display.show(Image.SAD)
        else:
            display.show(Image.HAPPY)
    elif channel == 1:
        led1 = value
        pin1.write_digital(led1)
        
def handle_actuators(in_buffer):
    
    in_pos = 0
    
    def get_byte() -> int:
        nonlocal in_pos
        byte = in_buffer[in_pos]
        in_pos += 1
        return byte

    def get_int() -> int:
        hi = get_byte()
        lo = get_byte()
        return hi * 256 + lo    

    in_pos = 0
    while in_pos < len(in_buffer):
        channel = get_byte()
        tag = get_byte()
        if (channel == 0 or channel == 1) and tag == LPP_dOut:
            handle_led(channel, get_byte())
        elif tag == LPP_dIn or tag == LPP_presence or tag == LPP_humidity:
            skip = get_byte()
        else:
            skip = get_int()        

def send_sensors(buttonA, buttonB):
    global counter
    
    out_buffer = bytearray(30)
    out_pos = 0
    
    def put_byte(value: int):
        nonlocal out_pos
        out_buffer[out_pos] = value
        out_pos += 1

    def put_int(value: int):
        while value < 0:
            value = value + 65536
        put_byte(value // 256)
        put_byte(value % 256)

    def put_sensor_byte(channel: int, tag: int, value: int):
        put_byte(channel)
        put_byte(tag)
        put_byte(value)
    
    def put_sensor_int(channel: int, tag: int, value: int):
        put_byte(channel)
        put_byte(tag)
        put_int(value)    
    
    out_pos = 0
    put_byte(uplink_tag) # header
    put_int(nodeID)
    put_int(counter)
    counter += 1
  
    put_sensor_byte(0, LPP_dOut, led0)
    put_sensor_byte(1, LPP_dOut, led1)
    put_sensor_byte(2, LPP_dIn, buttonA)
    put_sensor_byte(3, LPP_dIn, buttonB)
    put_sensor_int(4, LPP_temperature, temperature() * 10)
    put_sensor_int(8, LPP_aIn, display.read_light_level())

    print("send: " + str(list(out_buffer[:out_pos])))
    radio.send_bytes(out_buffer[:out_pos]) 
    
timer_period = 60000
timer_deadline = ticks_add(ticks_ms(), timer_period)

display.scroll(hex(nodeID))
print(hex(nodeID))

radio.on()
send_sensors(button_a.is_pressed(), button_b.is_pressed())
while True:
    if button_a.was_pressed():
        send_sensors(1, button_b.is_pressed())
        
    if button_b.was_pressed():
        send_sensors(button_a.is_pressed(), 1)
        
    if ticks_diff(timer_deadline, ticks_ms()) <= 0:
        send_sensors(button_a.is_pressed(), button_b.is_pressed())
        timer_deadline = ticks_add(timer_deadline, timer_period)
        
    rec_bytes = radio.receive_bytes()
    if rec_bytes != None:
        if rec_bytes[0] == downlink_tag:
            rec_nodeID = rec_bytes[1] * 256 + rec_bytes[2]
            if nodeID == rec_nodeID:  # msg for this node
                display.show('R')
                rec_counter = rec_bytes[3] * 256 + rec_bytes[4]
                handle_actuators(rec_bytes[5:])
                send_sensors(button_a.is_pressed(), button_a.is_pressed())

    sleep_ms(10)